### 모듈 import 하기

In [1]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
import re
import urllib.request


import nltk
from konlpy.tag import Okt
from nltk.tokenize import word_tokenize
okt = Okt()

from tqdm import tqdm
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [2]:
# from konlpy.tag import Hannanum

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/adminuser/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
# !pip install h5py --upgrade

Requirement already up-to-date: h5py in /anaconda/envs/version_test_azureml_py38/lib/python3.8/site-packages (3.6.0)


### 데이터 전처리
- 데이터셋 불러오기
- 원하는 데이터프레임으로 만들어주기

In [10]:
import pandas as pd
import numpy as np

df = pd.read_csv('./raw_data/일간베스트_댓글.txt',sep='|' , header=None, names= ['댓글', '비속어여부'])
df.sample(10)

,댓글,비속어여부
5013,어떤 놈는 선거에 맞춰서 남북정상회담도 조작질 해대는데 어떤 년은 선거가 코앞인데 ...,1
3659,용접은 ㅎㅌㅊ 직업.7등급은 ㅆㅎㅌㅊ 빡대가리임.ㅎㅌㅊ들과 ㅆㅎㅌㅊ들끼리 싸움을 붙...,1
4910,계정신고해라,0
2191,@노가다쟁이 />,1
309,세스코도 졵나 감당안되는곳은 그냥 물건 싹다버리라거나 포기함,1
5609,그냥 안봐도 보임좌파=돈앞에선 같은편이고 뭐고 없음,1
4681,자동차 라이센스 때문인가 ㅋㅋ,0
4120,앗!아까그새기잔어 개새기야,1
4007,전기차가 나중에 상용화 될때까지 기다려라 지금사면 병신테스터 일뿐이다.니네집 주차장...,1
3565,차차차~,0


In [12]:
swear = pd.read_csv('./raw_data/bad_words.txt')
swear.head()

,badwords
0,10새
1,10새기
2,10새리
3,10세리
4,10쉐이


In [13]:
swear['비속어여부']= 1

In [14]:
# swear['badwords'] = swear['댓글']
swear.rename(columns = {'badwords':'댓글'},inplace=True)

In [15]:
swear

,댓글,비속어여부
0,10새,1
1,10새기,1
2,10새리,1
3,10세리,1
4,10쉐이,1
...,...,...
1100,fuckyou,1
1101,nflavor,1
1102,penis,1
1103,pennis,1


In [16]:
print(swear.shape)
print(df.shape)

(1105, 2)
(5825, 2)


In [17]:
# 일베댓글과 욕설댓글 합치기
badcomm = pd.concat([df,swear])
print(badcomm.shape)

(6930, 2)


In [18]:
# 중복 댓글 확인

badcomm['댓글'].nunique(), badcomm['비속어여부'].nunique()

# 총 4개의 중복댓글, 비속어여부가 0과1 두가지가 아닌 4가지로 나옴

(6926, 4)

In [19]:
badcomm['비속어여부'].unique()
# 삭제해줘야 할 행

array(['1', '0',
       ' 한경닷컴 - 한국경제여자 형제가 인생을 더 행복하게 만든다 - 동아일보여자 형제 없는 남성을 선호한다?_채널A_웰컴투시월드 16회여자형제와 너무 친한 남친, 이건 정말 싫어 - 싱글즈여자형제 많은 사람의 특징이 뭔가요? - - ::: 알찬살림 요리정보 ...여자형제있는 사람이 눈치빠린거 같지않나요? - - ::: 알찬살림 ...여자 형제 있는 남돌과 없는 남돌의 차이 : 네이트판',
       1], dtype=object)

In [20]:
# 댓글 열의 중복 제거
badcomm.drop_duplicates(subset=['댓글'], inplace=True)

In [21]:
print('총 샘플의 수 :',len(badcomm))

총 샘플의 수 : 6926


In [22]:
badcomm.loc[(badcomm['비속어여부']!= '1') & (badcomm['비속어여부']!= '0')]

,댓글,비속어여부
455,여자 형제 둔 남성 보수 성향 강해… 美조사,한경닷컴 - 한국경제여자 형제가 인생을 더 행복하게 만든다 - 동아일보여자 형제 ...
0,10새,1
1,10새기,1
2,10새리,1
3,10세리,1
...,...,...
1100,fuckyou,1
1101,nflavor,1
1102,penis,1
1103,pennis,1


In [23]:
badcomm= badcomm.drop(455, axis=0)

In [24]:
badcomm.loc[(badcomm['비속어여부']!= 1) & (badcomm['비속어여부']!= 0)]

,댓글,비속어여부
0,좌배 까는건 ㅇㅂ,1
1,집에 롱 패딩만 세 개다. 10년 더 입어야지 ㅋㅋ,0
2,개소리야 니가 빨갱이를 옹호하고 드루킹을 ㅇㅇ짓이라고 말못해서 삐진거야 빨갱아,1
3,세탁이라고 봐도 된다,0
4,애새끼가 초딩도 아니고 ㅋㅋㅋㅋ,1
...,...,...
5820,좌우 헬파이어 3개씩 6개 장착에 아파치보다 약하지만 20mm 기관포 장착임,0
5821,"세금 내놓으라고 데모질 중 ㅋㅋ간첩, 도둑놈 새끼들이 대통령 해처먹으니까 나도 같...",1
5822,너가 한 말 중에,0
5823,제갈대중 ㅇㅂ,0


In [25]:
badcomm.dtypes

댓글       object
비속어여부    object
dtype: object

In [26]:
swear.dtypes

댓글       object
비속어여부     int64
dtype: object

In [27]:
df.dtypes

댓글       object
비속어여부    object
dtype: object

In [28]:
badcomm['비속어여부']=badcomm['비속어여부'].astype('int64')
# 비속어여부의 데이터타입을 int로 변경
badcomm['댓글']=badcomm['댓글'].astype('str')
# 댓글 데이터타입을 str으로 변경

In [29]:
# 중복 댓글 확인

badcomm['댓글'].nunique(), badcomm['비속어여부'].nunique()


(6924, 2)

In [30]:
print(badcomm.groupby('비속어여부').size().reset_index(name = 'count'))

   비속어여부  count
0      0   3780
1      1   3144


In [31]:
# 결측값 확인

print(badcomm.isnull().sum())

댓글       0
비속어여부    0
dtype: int64


In [32]:
# 한글과 공백을 제외하고 모두 제거
badcomm['댓글'] = badcomm['댓글'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
badcomm[:5]

,댓글,비속어여부
0,좌배 까는건 ㅇㅂ,1
1,집에 롱 패딩만 세 개다 년 더 입어야지 ㅋㅋ,0
2,개소리야 니가 빨갱이를 옹호하고 드루킹을 ㅇㅇ짓이라고 말못해서 삐진거야 빨갱아,1
3,세탁이라고 봐도 된다,0
4,애새끼가 초딩도 아니고 ㅋㅋㅋㅋ,1


In [34]:
# 특수문자, 이모티콘 제거

def preprocessing(sentence):
    sentence_split = sentence.split()
    sentence_split_result = []
    for word in sentence_split:
        if word[0] != '@' and word[:4] !='http':
            sentence_split_result.append(word)
    sentence_result = ' '.join(sentence_split_result)
    text_result = ''.join(re.compile('[가-힣|0-9|a-z| ]+').\
                    findall(sentence_result)).strip()
    return text_result

badcomm["댓글"] = badcomm['댓글'].apply(preprocessing)


In [35]:
badcomm[:15]

,댓글,비속어여부
0,좌배 까는건,1
1,집에 롱 패딩만 세 개다 년 더 입어야지,0
2,개소리야 니가 빨갱이를 옹호하고 드루킹을 짓이라고 말못해서 삐진거야 빨갱아,1
3,세탁이라고 봐도 된다,0
4,애새끼가 초딩도 아니고,1
5,부대의 후예라 그런지 가학적인 아이디어는 세계최고임 이래서 애교만 떨어도 돈 벌리는...,1
6,재앙이한건햇노,1
7,글쓴이 와꾸 승리에 비하면 방사능 피폭 원숭이 일듯,1
8,마 씨발련 아 몇평이고 맷개드갔노 니 대하이햄하고 해밨나,1
9,은행에 대출 상담 받으러 가보면 직업의 귀천 바로 알려줌,0


In [36]:
 # 기존에 한글이 없는 리뷰였다면 빈(empty) 값이 있을것
 # 공백(whitespace)만 있거나 빈 값을 가진 행이 있다면 Null 값으로 변경 후, Null 값이 존재하는지 확인하기

badcomm["댓글"] = badcomm["댓글"].str.replace('^ +', "") # white space 데이터를 empty value로 변경
badcomm["댓글"].replace('', np.nan, inplace=True)
print(badcomm.isnull().sum())

댓글       153
비속어여부      0
dtype: int64


In [37]:
# null 값 확인

badcomm.loc[badcomm.댓글.isnull()][:5]


,댓글,비속어여부
10,NaN,0
11,NaN,0
23,NaN,0
118,NaN,0
139,NaN,0


In [38]:
# null 값 제거

badcomm = badcomm.dropna(how = 'any')
print(len(badcomm))

6771


### 토큰화 - 불용어 제거

In [39]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [40]:
# 데이터셋 분리

text = badcomm['댓글'] # 시리즈 객체로 저장
score = badcomm['비속어여부']

### train, test 데이터 분리

In [41]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(text, score , test_size=0.2, random_state=0)
print(len(train_x), len(train_y), len(test_x), len(test_y))

5416 5416 1355 1355


In [42]:
train_x

2703               뒤에서 박는것 보다 차선 변경 사고가 안전빵이지
639                                         쎾
5378                                 올 가을쯤 되면
4540                        개일베잡짓거리하다 니인생종친거다
4289                      돈이없어서 좋은데 못살지 병신새끼야
                        ...                  
5018                              이럴땐 사이다 럼프형
3329                                   섹스자지보지
1690               원래 컴퓨터 바이러스도 백신이 다 못잡아 내는데
2661                민주당 드루킹댓글조작알바 새끼들 정신 못차렸네
2787    디지털이 발전하면서 예견됐던 일이지 근데 여자가 먼저 찍자고 하던데
Name: 댓글, Length: 5416, dtype: object

In [43]:
# 형태소 분석기를 사용하여 토큰화를 하면서 불용어를 제거하여 X_train에 저장

X_train = []
for sentence in tqdm(train_x):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화, 동사 기본형으로 변경
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    X_train.append(stopwords_removed_sentence)

100%|██████████| 5416/5416 [00:15<00:00, 346.35it/s]


In [44]:
print(X_train[:3])

[['뒤', '에서', '박다', '보다', '차선', '변경', '사고', '안전', '빵', '이지'], ['쎾'], ['오다', '가을', '쯤', '되다']]


In [45]:
# 형태소 분석기를 사용하여 토큰화를 하면서 불용어를 제거하여 X_test에 저장

X_test = []
for sentence in tqdm(test_x):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화, 동사 기본형으로 변경
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    X_test.append(stopwords_removed_sentence)

100%|██████████| 1355/1355 [00:02<00:00, 502.81it/s]


In [46]:
print(X_test[:3])

[['쉬빨'], ['모노노케', '히', '메'], ['수셔']]


### 정수 인코딩

In [47]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
print(tokenizer.word_index)

{'있다': 1, '을': 2, '보다': 3, '되다': 4, '아니다': 5, '없다': 6, '다': 7, '새끼': 8, '같다': 9, '로': 10, '에서': 11, '안': 12, '아': 13, '고': 14, '만': 15, '못': 16, '그렇다': 17, '나': 18, '이다': 19, '말': 20, '저': 21, '그': 22, '거': 23, '년': 24, '먹다': 25, '개': 26, '사람': 27, '니': 28, '임': 29, '존나': 30, '게': 31, '들다': 32, '맞다': 33, '내': 34, '가다': 35, '하고': 36, '때': 37, '지': 38, '돈': 39, '야': 40, '그냥': 41, '좋다': 42, '병신': 43, '면': 44, '진짜': 45, '네': 46, '일': 47, '더': 48, '애': 49, '적': 50, '생각': 51, '라': 52, '받다': 53, '오다': 54, '뭐': 55, '것': 56, '나오다': 57, '왜': 58, '인': 59, '치다': 60, '인데': 61, '용접': 62, '글': 63, '않다': 64, '좆': 65, '한테': 66, '근데': 67, '알다': 68, '기': 69, '놈': 70, '많다': 71, '이나': 72, '씹다': 73, '주다': 74, '함': 75, '서': 76, '정도': 77, '지다': 78, '이냐': 79, '너': 80, '한국': 81, '씨발': 82, '냐': 83, '모르다': 84, '소리': 85, '쓰다': 86, '사': 87, '하': 88, '나다': 89, '지금': 90, '여자': 91, '랑': 92, '까지': 93, '일본': 94, '살다': 95, '나라': 96, '수': 97, '지랄': 98, '일베': 99, '안되다': 100, '시발': 101, '딱': 102, '공부': 103, '많이': 104, '이지':

In [48]:
# 빈도수가 낮은 단어들은 자연어 처리에서 배제
# 등장 빈도수가 3회 미만인 단어들이 이 데이터에서 얼만큼의 비중을 차지하는지 확인

threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

# 생각보다 희귀단어 등장빈도 비율이 높다.

단어 집합(vocabulary)의 크기 : 9847
등장 빈도가 2번 이하인 희귀 단어의 수: 6992
단어 집합에서 희귀 단어의 비율: 71.00639788768153
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 17.015263985352945


In [49]:
# 등장 빈도수가 2이하인 단어들의 수를 제외한 단어의 개수를 단어 집합의 최대 크기로 제한

# 전체 단어 개수 중 빈도수 2이하인 단어는 제거.
# 0번 패딩 토큰을 고려하여 + 1
vocab_size = total_cnt - rare_cnt + 1
print('단어 집합의 크기 :',vocab_size)

단어 집합의 크기 : 2856


In [50]:
# 케라스 토크나이저의 인자로 넘겨주고 텍스트 시퀀스를 정수 시퀀스로 변환

tokenizer = Tokenizer(vocab_size) 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [51]:
# 정수 인코딩이 잘 되었나?
print(X_train[:3])

[[468, 11, 654, 3, 2180, 1434, 1090, 962, 1091, 105], [], [54, 655, 4]]


In [52]:
print(train_y)

2703    0
639     1
5378    0
4540    1
4289    1
       ..
5018    0
3329    1
1690    0
2661    1
2787    0
Name: 비속어여부, Length: 5416, dtype: int64


In [53]:
print(test_y)

816     1
2360    0
808     1
460     1
692     1
       ..
1090    1
5650    0
3914    0
33      0
2829    0
Name: 비속어여부, Length: 1355, dtype: int64


In [54]:
# y를 array 형식으로 저장해주기

train_y = np.array(train_y)
test_y = np.array(test_y)

In [55]:
print(train_y)
print(test_y )

[0 1 0 ... 0 1 0]
[1 0 1 ... 0 0 0]


### 패딩으로 댓글 길이 맞추기

In [56]:
print('리뷰의 최대 길이 :',max(len(review) for review in train_x))
print('리뷰의 평균 길이 :',sum(map(len, train_x))/len(train_x))

# 최대길이가 요상한 댓글이 있음

리뷰의 최대 길이 : 911
리뷰의 평균 길이 : 26.43519202363368


In [57]:
def below_threshold_len(max_len, nested_list):
  count = 0
  for sentence in nested_list:
    if(len(sentence) <= max_len):
        count = count + 1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (count / len(nested_list))*100))

In [60]:
max_len = 30
below_threshold_len(max_len, X_train)

# 96 % 가 30 이하이기 때문에 최대글자수는 30으로 패딩해주는것이 좋을것같다.

전체 샘플 중 길이가 30 이하인 샘플의 비율: 96.49187592319055


In [68]:
# 모든 샘플의 길이를 30으로 맞춘다 - 이때 정수인코딩한 데이터 사용!
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

### LSTM으로 분류하기
#### 하이퍼파라미터
- 임베딩 벡터 차원 = 100
- 은닉 상태 크기 = 128
- 배치 크기 = 64
- 에포크 = 15

In [70]:
# 두 개의 선택지 중 하나를 예측하는 이진 분류 문제를 수행하는 모델    

from tensorflow.python.keras.layers import Embedding, Dense, LSTM
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.models import load_model
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint # 조기종료

embedding_dim = 100
hidden_units = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(1, activation='sigmoid')) # 로지스틱 회귀를 사용해야 하기 때문에 sigmoid

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, train_y, epochs=15, callbacks=[es, mc], batch_size=64, validation_split=0.2)

2022-05-01 13:21:16.506619: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-01 13:21:17.184298: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6589 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0001:00:00.0, compute capability: 6.0


Epoch 1/15
68/68 [==============================] - 15s 113ms/step - loss: 0.6438 - acc: 0.6383 - val_loss: 0.5743 - val_acc: 0.7445

Epoch 00001: val_acc improved from -inf to 0.74446, saving model to best_model.h5
Epoch 2/15
68/68 [==============================] - 7s 104ms/step - loss: 0.4719 - acc: 0.7902 - val_loss: 0.5122 - val_acc: 0.7546

Epoch 00002: val_acc improved from 0.74446 to 0.75461, saving model to best_model.h5
Epoch 3/15
68/68 [==============================] - 7s 104ms/step - loss: 0.3602 - acc: 0.8495 - val_loss: 0.4951 - val_acc: 0.8072

Epoch 00003: val_acc improved from 0.75461 to 0.80720, saving model to best_model.h5
Epoch 4/15
68/68 [==============================] - 7s 104ms/step - loss: 0.2931 - acc: 0.8843 - val_loss: 0.5209 - val_acc: 0.7961

Epoch 00004: val_acc did not improve from 0.80720
Epoch 5/15
68/68 [==============================] - 7s 104ms/step - loss: 0.2514 - acc: 0.8968 - val_loss: 0.5477 - val_acc: 0.7961

Epoch 00005: val_acc did not imp

In [71]:
# 모델 정확도 테스트
loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, test_y)[1]))

43/43 [==============================] - 1s 11ms/step - loss: 0.5089 - acc: 0.7963

 테스트 정확도: 0.7963


### 모델 사용해서 댓글 비속어 포함여부 예측해보기

In [72]:
def sentiment_predict(new_sentence):
  new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
  new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화
  new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
  encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
  pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
  score = float(loaded_model.predict(pad_new)) # 예측
  if(score > 0.5):
    print("{:.2f}% 확률로 비속어가 포함된 댓글입니다.\n".format(score * 100))
  else:
    print("{:.2f}% 확률로 일반 댓글입니다.\n".format((1 - score) * 100))

In [73]:
sentiment_predict('마 씨발련 아 몇평이고 맷개드갔노 니 대하이햄하고 해밨나')

90.43% 확률로 비속어가 포함된 댓글입니다.



In [75]:
sentiment_predict('우리지역군데 금태섭 뽑으면 안되지 그래도 자한당 뻡아야겠지')

99.87% 확률로 일반 댓글입니다.



In [76]:
sentiment_predict('애새끼가 초딩도 아니고')

98.58% 확률로 비속어가 포함된 댓글입니다.



In [77]:
sentiment_predict('재앙이한건햇노')

68.24% 확률로 비속어가 포함된 댓글입니다.

